# SQL Profit Aggregates - Interactive Analysis

This notebook demonstrates the four SQL profit aggregate queries from the `sql_profit_aggregates.py` module.

The queries provide different perspectives on profit analysis:
1. **Profit by Year** - Yearly profit trends
2. **Profit by Year + Category** - Category performance by year
3. **Profit by Customer** - Top and bottom customers
4. **Profit by Customer + Year** - Customer profitability over time

## 1. Import Required Libraries

In [1]:
import sys
import os
sys.path.insert(0, '/Users/kushalsenlaskar/Documents/E-commerce Sales Data/src')

from pyspark.sql import SparkSession
import pandas as pd

from sql_profit_aggregates import (
    get_profit_by_year,
    get_profit_by_year_category,
    get_profit_by_customer,
    get_profit_by_customer_year,
    get_all_profit_aggregates,
    validate_all_aggregates
)

from load_source_data import load_customer_data, load_orders_data, load_products_data
from data_cleaning_utils import (
    clean_orders_for_enrichment,
    clean_customers_for_enrichment,
    clean_products_for_enrichment
)
from load_enriched_orders import create_enriched_orders_table

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Initialize Spark Session and Load Data

In [2]:
# Create Spark session
spark = SparkSession.builder \
    .appName("SQL_Profit_Aggregates") \
    .master("local[*]") \
    .getOrCreate()

print("Spark session created!")

# Define data paths
project_root = '/Users/kushalsenlaskar/Documents/E-commerce Sales Data'
customer_path = os.path.join(project_root, 'data', 'Customer.xlsx')
orders_path = os.path.join(project_root, 'data', 'Orders.json')
products_path = os.path.join(project_root, 'data', 'Products.csv')

# Load raw source data
print("\nLoading raw source data...")
customers_df = load_customer_data(spark, customer_path)
orders_df = load_orders_data(spark, orders_path)
products_df = load_products_data(spark, products_path)

print(f"Raw data loaded:")
print(f"  Customers: {customers_df.count()} records")
print(f"  Orders: {orders_df.count()} records")
print(f"  Products: {products_df.count()} records")

# Apply data cleaning
print("\n=== Applying Data Cleaning ===")
customers_df = clean_customers_for_enrichment(customers_df)
orders_df = clean_orders_for_enrichment(orders_df)
products_df = clean_products_for_enrichment(products_df)

print(f"\nCleaned data:")
print(f"  Customers: {customers_df.count()} records")
print(f"  Orders: {orders_df.count()} records")
print(f"  Products: {products_df.count()} records")

25/11/30 01:57:46 WARN Utils: Your hostname, Kushals-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.136 instead (on interface en0)
25/11/30 01:57:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/30 01:57:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/30 01:57:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/30 01:57:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/30 01:57:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/11/30 01:57:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/30 01:57:47 WARN Utils: Service 

Spark session created!

Loading raw source data...

Checking file at: /Users/kushalsenlaskar/Documents/E-commerce Sales Data/data/Customer.xlsx
File found. Loading Excel data using Spark...
Customer data loaded successfully

Checking file at: /Users/kushalsenlaskar/Documents/E-commerce Sales Data/data/Orders.json
File found. Loading JSON data using Spark...
Orders data loaded successfully

Checking file at: /Users/kushalsenlaskar/Documents/E-commerce Sales Data/data/Products.csv
File found. Loading CSV data using Spark...
Products data loaded successfully
Raw data loaded:
Customer data loaded successfully

Checking file at: /Users/kushalsenlaskar/Documents/E-commerce Sales Data/data/Orders.json
File found. Loading JSON data using Spark...
Orders data loaded successfully

Checking file at: /Users/kushalsenlaskar/Documents/E-commerce Sales Data/data/Products.csv
File found. Loading CSV data using Spark...
Products data loaded successfully
Raw data loaded:


  Customers: 793 records
  Orders: 9994 records
  Products: 1851 records

=== Applying Data Cleaning ===

--- Cleaning Customers Data for Enrichment ---
  Orders: 9994 records
  Products: 1851 records

=== Applying Data Cleaning ===

--- Cleaning Customers Data for Enrichment ---
Removing records with NULL Customer ID...
   Records with NULL Customer ID: 0
Removing duplicate Customer IDs...
Removing records with NULL Customer ID...
   Records with NULL Customer ID: 0
Removing duplicate Customer IDs...
   Records with duplicate Customer ID removed: 0
   Records with duplicate Customer ID removed: 0
Customers data cleaning completed
  Original records: 793, After cleaning: 793
  Total records removed: 0

--- Cleaning Orders Data for Enrichment ---
Removing records with negative Profit...
Customers data cleaning completed
  Original records: 793, After cleaning: 793
  Total records removed: 0

--- Cleaning Orders Data for Enrichment ---
Removing records with negative Profit...
   Records 

## 3. Create Enriched Orders Table

In [3]:
# Create enriched orders table
print("Creating enriched orders table...")
enriched_orders_df = create_enriched_orders_table(orders_df, customers_df, products_df)

print(f"Enriched orders table created: {enriched_orders_df.count()} records")
print(f"\nColumns: {enriched_orders_df.columns}")

Creating enriched orders table...

--- Cleaning Orders Data for Enrichment ---
Removing records with negative Profit...
   Records with negative Profit: 0
Removing records with NULL Order ID...
   Records with NULL Order ID: 0
   Removing duplicate Order IDs...
   Records with negative Profit: 0
Removing records with NULL Order ID...
   Records with NULL Order ID: 0
   Removing duplicate Order IDs...
   Records with duplicate Order ID removed: 0
Validating date formats...
   Records with duplicate Order ID removed: 0
Validating date formats...
   Records with invalid date format: 0
Removing records with NULL Customer ID...
   Records with invalid date format: 0
Removing records with NULL Customer ID...
   Records with NULL Customer ID: 0
Removing records with NULL Product ID...
   Records with NULL Customer ID: 0
Removing records with NULL Product ID...
   Records with NULL Product ID: 0
   Records with NULL Product ID: 0
Orders data cleaning completed
  Original records: 4427, After c

## 4. Generate All Profit Aggregates

In [4]:
# Generate all four profit aggregates
print("Generating profit aggregates...\n")
aggregates = get_all_profit_aggregates(enriched_orders_df)

print("Aggregates generated successfully!")
print(f"Available aggregates: {list(aggregates.keys())}")

Generating profit aggregates...

Aggregates generated successfully!
Available aggregates: ['profit_by_year', 'profit_by_year_category', 'profit_by_customer', 'profit_by_customer_year']
Aggregates generated successfully!
Available aggregates: ['profit_by_year', 'profit_by_year_category', 'profit_by_customer', 'profit_by_customer_year']


25/11/30 01:58:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## 5. Query 1: Profit by Year

In [5]:
# Display profit by year
profit_by_year = aggregates['profit_by_year']

print("\n--- Profit by Year ---")
profit_by_year.show()

# Convert to pandas for analysis
pdf_year = profit_by_year.toPandas()
print(f"\nSummary Statistics:")
print(pdf_year.describe())


--- Profit by Year ---
+----+------------+-----------+
|Year|Total_Profit|Order_Count|
+----+------------+-----------+
|2014|     39104.9|        830|
|2015|     46621.2|        902|
|2016|    64079.74|       1147|
|2017|     69547.6|       1448|
+----+------------+-----------+

+----+------------+-----------+
|Year|Total_Profit|Order_Count|
+----+------------+-----------+
|2014|     39104.9|        830|
|2015|     46621.2|        902|
|2016|    64079.74|       1147|
|2017|     69547.6|       1448|
+----+------------+-----------+


Summary Statistics:
              Year  Total_Profit  Order_Count
count     4.000000      4.000000     4.000000
mean   2015.500000  54838.360000  1081.750000
std       1.290994  14339.087927   279.335372
min    2014.000000  39104.900000   830.000000
25%    2014.750000  44742.125000   884.000000
50%    2015.500000  55350.470000  1024.500000
75%    2016.250000  65446.705000  1222.250000
max    2017.000000  69547.600000  1448.000000

Summary Statistics:
      

## 6. Visualize Profit by Year

In [6]:
# Convert to pandas and display analysis
pdf_year = aggregates['profit_by_year'].toPandas()

print("\nProfit by Year Summary:")
print(pdf_year.to_string(index=False))

print(f"\nBasic Statistics:")
print(f"  Total Profit: ${pdf_year['Total_Profit'].sum():,.2f}")
print(f"  Average Annual Profit: ${pdf_year['Total_Profit'].mean():,.2f}")
print(f"  Max Annual Profit: ${pdf_year['Total_Profit'].max():,.2f}")
print(f"  Min Annual Profit: ${pdf_year['Total_Profit'].min():,.2f}")


Profit by Year Summary:
 Year  Total_Profit  Order_Count
 2014      39104.90          830
 2015      46621.20          902
 2016      64079.74         1147
 2017      69547.60         1448

Basic Statistics:
  Total Profit: $219,353.44
  Average Annual Profit: $54,838.36
  Max Annual Profit: $69,547.60
  Min Annual Profit: $39,104.90


## 7. Query 2: Profit by Year + Category

In [7]:
# Display profit by year and category
profit_by_year_category = aggregates['profit_by_year_category']

print("\n--- Profit by Year + Category ---")
profit_by_year_category.show(20)

# Convert to pandas
pdf_year_cat = profit_by_year_category.toPandas()
print(f"\nTotal rows: {len(pdf_year_cat)}")
print(f"\nCategories: {pdf_year_cat['Category'].unique()}")


--- Profit by Year + Category ---
+----+---------------+------------+-----------+
|Year|       Category|Total_Profit|Order_Count|
+----+---------------+------------+-----------+
|2014|      Furniture|     7395.89|        140|
|2014|Office Supplies|    15850.52|        524|
|2014|     Technology|    15858.49|        166|
|2015|      Furniture|    11258.47|        161|
|2015|Office Supplies|    15524.48|        539|
|2015|     Technology|    19838.25|        202|
|2016|      Furniture|    10429.55|        214|
|2016|Office Supplies|    23122.29|        712|
|2016|     Technology|     30527.9|        221|
|2017|      Furniture|     8895.09|        227|
|2017|Office Supplies|    33224.52|        939|
|2017|     Technology|    27427.99|        282|
+----+---------------+------------+-----------+

+----+---------------+------------+-----------+
|Year|       Category|Total_Profit|Order_Count|
+----+---------------+------------+-----------+
|2014|      Furniture|     7395.89|        140|
|201

## 8. Visualize Profit by Year + Category

In [8]:
# Display profit by year and category in detail
pdf_year_cat = aggregates['profit_by_year_category'].toPandas()

print("\nProfit by Year and Category Summary:")
print(pdf_year_cat.to_string(index=False))

print(f"\nTotal unique Year-Category combinations: {len(pdf_year_cat)}")

# Summary by category
print("\nTotal Profit by Category:")
category_summary = pdf_year_cat.groupby('Category')['Total_Profit'].sum().sort_values(ascending=False)
for category, profit in category_summary.items():
    print(f"  {category}: ${profit:,.2f}")


Profit by Year and Category Summary:
 Year        Category  Total_Profit  Order_Count
 2014       Furniture       7395.89          140
 2014 Office Supplies      15850.52          524
 2014      Technology      15858.49          166
 2015       Furniture      11258.47          161
 2015 Office Supplies      15524.48          539
 2015      Technology      19838.25          202
 2016       Furniture      10429.55          214
 2016 Office Supplies      23122.29          712
 2016      Technology      30527.90          221
 2017       Furniture       8895.09          227
 2017 Office Supplies      33224.52          939
 2017      Technology      27427.99          282

Total unique Year-Category combinations: 12

Total Profit by Category:
  Technology: $93,652.63
  Office Supplies: $87,721.81
  Furniture: $37,979.00


## 9. Query 3: Profit by Customer

In [9]:
# Display profit by customer
profit_by_customer = aggregates['profit_by_customer']

print("\n--- Top 15 Customers by Profit ---")
profit_by_customer.limit(15).show()

pdf_customer = profit_by_customer.toPandas()
print(f"\nTotal customers: {len(pdf_customer)}")
print(f"\nTop customer: {pdf_customer.iloc[0]['Customer Name']} (${pdf_customer.iloc[0]['Total_Profit']:.2f})")
print(f"Bottom customer: {pdf_customer.iloc[-1]['Customer Name']} (${pdf_customer.iloc[-1]['Total_Profit']:.2f})")


--- Top 15 Customers by Profit ---
+----------------+------------+-----------+
|   Customer Name|Total_Profit|Order_Count|
+----------------+------------+-----------+
|    Tamara Chand|     8443.12|          4|
|   Adrian Barton|      5800.2|          9|
|    Hunter Lopez|     5185.18|          5|
|    Sanjit Engle|     2916.79|          9|
|   Karen Dan!els|     2592.29|          5|
|Tom Boeckenhauer|      2283.8|          7|
|       Jane Waco|     2115.66|          6|
|             NaN|      2108.6|         52|
|    Fred Hopkins|     1772.47|          7|
|       Pete Kriz|     1666.61|         11|
|     John Murray|      1559.6|          5|
|  Alan Dominguez|     1551.73|          7|
|Corinna Mitchell|     1510.51|          6|
|   Yana Sorensen|     1450.42|          7|
|  Lena Creighton|     1430.01|          9|
+----------------+------------+-----------+

+----------------+------------+-----------+
|   Customer Name|Total_Profit|Order_Count|
+----------------+------------+--------

## 10. Visualize Top Customers

In [10]:
# Top 10 and Bottom 10 customers analysis
pdf_customer = aggregates['profit_by_customer'].toPandas()
top_10 = pdf_customer.head(10)
bottom_10 = pdf_customer.tail(10)

print("\n--- Top 10 Customers by Profit ---")
print(top_10.to_string(index=False))

print("\n--- Bottom 10 Customers by Profit ---")
print(bottom_10.to_string(index=False))

print(f"\nProfit Range:")
print(f"  Highest: ${top_10.iloc[0]['Total_Profit']:,.2f}")
print(f"  Lowest: ${bottom_10.iloc[-1]['Total_Profit']:,.2f}")
print(f"  Difference: ${top_10.iloc[0]['Total_Profit'] - bottom_10.iloc[-1]['Total_Profit']:,.2f}")


--- Top 10 Customers by Profit ---
   Customer Name  Total_Profit  Order_Count
    Tamara Chand       8443.12            4
   Adrian Barton       5800.20            9
    Hunter Lopez       5185.18            5
    Sanjit Engle       2916.79            9
   Karen Dan!els       2592.29            5
Tom Boeckenhauer       2283.80            7
       Jane Waco       2115.66            6
             NaN       2108.60           52
    Fred Hopkins       1772.47            7
       Pete Kriz       1666.61           11

--- Bottom 10 Customers by Profit ---
   Customer Name  Total_Profit  Order_Count
   Randy Bradley          5.09            2
   Evan Minnotte          4.00            1
  Susan Gilcrest          3.61            2
   Pauline Chand          3.36            1
 Mitch Gastineau          1.85            1
  Michael Oakman          1.80            1
    Carl Jackson          1.65            1
     Jim Radford          1.58            2
Pete@#$ Takahito          1.11            1
 

## 11. Query 4: Profit by Customer + Year

In [11]:
# Display profit by customer and year
profit_by_customer_year = aggregates['profit_by_customer_year']

print("\n--- Profit by Customer + Year (Sample) ---")
profit_by_customer_year.limit(20).show()

pdf_cust_year = profit_by_customer_year.toPandas()
print(f"\nTotal records: {len(pdf_cust_year)}")


--- Profit by Customer + Year (Sample) ---
+--------------------+----+------------+-----------+
|       Customer Name|Year|Total_Profit|Order_Count|
+--------------------+----+------------+-----------+
|             Doro...|2014|        6.63|          1|
|         =--Katha...|2014|       19.08|          3|
|         Helen Ab...|2014|       108.8|          1|
|       Kristi;'[]...|2014|      139.22|          2|
|     Rac5467hel P...|2014|       11.76|          1|
|   _Mike Vitt 123...|2014|         6.9|          1|
|    Shahi  Collister|2014|         0.0|          1|
|   &&Tracy Blumstein|2014|       32.89|          2|
|      Shahi  Hopkins|2014|      105.25|          1|
|     Shahi  Shariari|2014|        9.32|          2|
|":[]|}-=Charles C...|2014|       50.33|          1|
|&^*(5678Shirley D...|2014|       55.52|          4|
|        )(*&Sung Pak|2014|        8.67|          1|
|     5467Ben Wallace|2014|       37.05|          2|
|      6789Erin  Mull|2014|       57.59|          1|
| 

## 12. Analyze Customer Trends Over Time

In [12]:
# Get top 5 customers and analyze trends
pdf_customer = aggregates['profit_by_customer'].toPandas()
top_5_customers = pdf_customer.head(5)['Customer Name'].tolist()

print("Top 5 Customers and Their Profit Trends Over Years:")
print("=" * 80)

pdf_cust_year = aggregates['profit_by_customer_year'].toPandas()

for customer in top_5_customers:
    customer_data = pdf_cust_year[pdf_cust_year['Customer Name'] == customer].sort_values('Year')
    print(f"\n{customer}:")
    print(customer_data[['Year', 'Total_Profit', 'Order_Count']].to_string(index=False))
    
    total_customer_profit = customer_data['Total_Profit'].sum()
    print(f"  Total Profit (All Years): ${total_customer_profit:,.2f}")

Top 5 Customers and Their Profit Trends Over Years:

Tamara Chand:
 Year  Total_Profit  Order_Count
 2014         11.72            1
 2015         28.86            1
 2016       8402.54            2
  Total Profit (All Years): $8,443.12

Adrian Barton:
 Year  Total_Profit  Order_Count
 2014        497.00            1
 2015         33.59            1
 2016       4952.27            2
 2017        317.34            5
  Total Profit (All Years): $5,800.20

Hunter Lopez:
 Year  Total_Profit  Order_Count
 2014         10.78            1
 2016        128.54            2
 2017       5045.86            2
  Total Profit (All Years): $5,185.18

Sanjit Engle:
 Year  Total_Profit  Order_Count
 2014         12.05            3
 2015         34.91            1
 2016       2806.85            2
 2017         62.98            3
  Total Profit (All Years): $2,916.79

Karen Dan!els:
 Year  Total_Profit  Order_Count
 2014         16.70            1
 2015        128.97            1
 2016       2446.62       

## 13. Validate All Aggregates

In [13]:
# Validate all aggregates
print("\n=== Validation Results ===")
validation_results = validate_all_aggregates(aggregates)

print("\nValidation Summary:")
for aggregate, is_valid in validation_results.items():
    status = "PASS" if is_valid else "FAIL"
    print(f"{aggregate}: {status}")

all_valid = all(validation_results.values())
print(f"\nAll aggregates valid: {all_valid}")


=== Validation Results ===

--- Validating Profit By Year ---
Profit By Year validation passed (4 rows)

--- Validating Profit By Year Category ---
Profit By Year validation passed (4 rows)

--- Validating Profit By Year Category ---
Profit By Year Category validation passed (12 rows)

--- Validating Profit By Customer ---
Profit By Year Category validation passed (12 rows)

--- Validating Profit By Customer ---
Profit By Customer validation passed (785 rows)

--- Validating Profit By Customer Year ---
Profit By Customer validation passed (785 rows)

--- Validating Profit By Customer Year ---
Profit By Customer Year validation passed (2321 rows)

Validation Summary:
profit_by_year: PASS
profit_by_year_category: PASS
profit_by_customer: PASS
profit_by_customer_year: PASS

All aggregates valid: True
Profit By Customer Year validation passed (2321 rows)

Validation Summary:
profit_by_year: PASS
profit_by_year_category: PASS
profit_by_customer: PASS
profit_by_customer_year: PASS

All aggr

## 14. Summary Statistics

In [14]:
# Calculate summary statistics
print("\n=== Summary Statistics ===")

pdf_year = aggregates['profit_by_year'].toPandas()
total_profit = pdf_year['Total_Profit'].sum()
total_orders = pdf_year['Order_Count'].sum()
avg_profit_per_year = pdf_year['Total_Profit'].mean()

print(f"\nProfit Metrics:")
print(f"  Total Profit: ${total_profit:,.2f}")
print(f"  Total Orders: {int(total_orders):,}")
print(f"  Average Profit per Year: ${avg_profit_per_year:,.2f}")

pdf_customer = aggregates['profit_by_customer'].toPandas()
print(f"\nCustomer Metrics:")
print(f"  Total Customers: {len(pdf_customer)}")
print(f"  Average Customer Profit: ${pdf_customer['Total_Profit'].mean():,.2f}")
print(f"  Max Customer Profit: ${pdf_customer['Total_Profit'].max():,.2f}")
print(f"  Min Customer Profit: ${pdf_customer['Total_Profit'].min():,.2f}")

pdf_year_cat = aggregates['profit_by_year_category'].toPandas()
print(f"\nCategory Metrics:")
category_summary = pdf_year_cat.groupby('Category')['Total_Profit'].sum().sort_values(ascending=False)
for category, profit in category_summary.items():
    print(f"  {category}: ${profit:,.2f}")


=== Summary Statistics ===

Profit Metrics:
  Total Profit: $219,353.44
  Total Orders: 4,327
  Average Profit per Year: $54,838.36

Customer Metrics:
  Total Customers: 785
  Average Customer Profit: $279.43
  Max Customer Profit: $8,443.12
  Min Customer Profit: $0.46

Category Metrics:
  Technology: $93,652.63
  Office Supplies: $87,721.81
  Furniture: $37,979.00

Profit Metrics:
  Total Profit: $219,353.44
  Total Orders: 4,327
  Average Profit per Year: $54,838.36

Customer Metrics:
  Total Customers: 785
  Average Customer Profit: $279.43
  Max Customer Profit: $8,443.12
  Min Customer Profit: $0.46

Category Metrics:
  Technology: $93,652.63
  Office Supplies: $87,721.81
  Furniture: $37,979.00


## 15. Cleanup

In [15]:
# Stop Spark session
spark.stop()
print("Spark session stopped!")

Spark session stopped!
